In [ ]:
import numpy as np
from datascience import *

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
# Fix for datascience plots
import collections as collections
import collections.abc as abc
collections.Iterable = abc.Iterable

# Python Dictionaries

It is time to learn another data structure built into Python -- dictionaries. With lists and arrays, you accessed elements index number.

In [ ]:
my_list = ['Temple University', 2023, np.pi]
my_list

In [ ]:
# Accessing the second element in my_list
my_list[1]

In [ ]:
my_array = make_array(1, 2, 3, 5, 7, 11, 13)
my_array

In [ ]:
# Accessing the second element in my_array
my_array[1]

By contrast, a dictionary stores any number of {key: value, key: value} pairs. You access a value in a dictionary by using its key. Dictionaries are defined using curly brackets. Notice that the values of a dictionary can be other data structures, such as a list.

In [ ]:
cst = {
    "name": "College of Science and Technology",
    "departments": ["Biology", "Chemistry", "CIS", "Math", "EES", "Physics"],
    "dean": "Michael Klein",
}
cst

In [ ]:
# Access a dictionary element by key
print(cst['dean'])
print(cst['departments'])

In [ ]:
cst.keys()

In [ ]:
cst.values()

Each key in a dictionary my be unique -- no duplicates. You can change a dictionary value simply by assigned a new value to a key.

In [ ]:
cst['dean'] = 'Miguel Mustafa'
cst

# Review of slicing a sequence - Tip for Lab 6

In [ ]:
seq = "ATCTTGGCTCTTTGCATCGGATGGAAGGACGCAGCGAAATGCGAT"
seq

In [ ]:
len(seq)

In [ ]:
seq[0:5]

In [ ]:
seq[:5]

In [ ]:
seq[-1]

In [ ]:
seq[40:-1]

In [ ]:
seq[0:10]

In [ ]:
i = 3
samplesize = 5
seq[i:i + samplesize]

In [ ]:
seq[0:10].count('G') 

# Central Limit Theorem

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
population_size = 1000000
population = np.random.rand(1000000)

In [ ]:
number_of_samples = 10000
sample_means = np.random.rand(number_of_samples)

# Watch what happens as we grow the sample size.
sample_size = 1

In [ ]:
c = np.random.rand(number_of_samples)
for i in range(0,number_of_samples):
 c = np.random.randint(1,population_size,sample_size)
 sample_means[i] = population[c].mean()

In [ ]:
plt.subplot(1,2,1)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
sns.histplot(sample_means,bins=int(180/5))
plt.title('Histogram of Sample mean',fontsize=20)
plt.xlabel('Sample mean',fontsize=20)
plt.ylabel('Count',fontsize=20)
plt.subplot(1,2,2)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
sns.kdeplot(sample_means)
plt.title('Density of Sample mean',fontsize=20)
plt.xlabel('Sample mean',fontsize=20)
plt.ylabel('Density',fontsize=20)
plt.subplots_adjust(bottom=0.1, right=2, top=0.9)